<a href="https://colab.research.google.com/github/marteczkah/guided_research/blob/main/Centroids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np  
import matplotlib.pyplot as plt  
import h5py
import plotly.express as px

-> add centroids to the data to the h5 file as 'centroid'

-> make it a 10000x1 shape -> each point has one centroid assigned -> can be changed later just an easy thing to implement for now

-> current data: <KeysViewHDF5 ['gt_label', 'gt_mask', 'gt_other_mask', 'gt_valid', 'pts']>

In [65]:
# start with testing everything on one chair
chair_gt = h5py.File("/content/gdrive/MyDrive/GR_Data/ins_seg_h5_for_detection/Chair-1/train-00.h5")

# Centroid Calculations

Centroid Calculation -> Function per Part Instance

In [66]:
def calculate_centroid_average(points):
  x = np.average(points[:, 0])
  y = np.average(points[:, 1])
  z = np.average(points[:, 2])
  return np.array([x, y, z])

In [67]:
def calculate_centroid_middle(points):
  x_min = np.min(points[:, 0])
  x_max = np.max(points[:, 0])
  y_min = np.min(points[:, 1])
  y_max = np.max(points[:, 1])
  z_min = np.min(points[:, 2])
  z_max = np.max(points[:, 2])
  return np.array([(x_min+x_max)/2, (y_min+y_max)/2, (z_min+z_max)/2])

# Visualization functions

Function to visualize part instances

In [68]:
def plot_cloud_instance(points, label_mask):
  my_list = []
  label_list = []
  for i, row in enumerate(label_mask):
    cur_inds = np.where(row)
    if len(cur_inds) > 0:
      for j in range(len(cur_inds[0])):
        cur_dic = {'num': cur_inds[0][j], 'label': i}
        label_list.append(cur_dic)
  for i, point in enumerate(points):
    x = point[0]
    y = point[1]
    z = point[2]
    # label = label_list[str(i)]
    for lab in label_list:
      if lab['num'] == i:
        label = lab['label']
    cur_dic = {'x': x, 'y': y, 'z': z, 'label': label}
    my_list.append(cur_dic)
  import pandas as pd
  data_frame = pd.DataFrame(my_list)  
  import plotly.express as px
  fig = px.scatter_3d(data_frame, x='z', y='x', z='y', color='label')
  fig.show()

Plot centroid on one part

In [69]:
def plot_centroid(points, centroid):
  my_list = []
  for i, point in enumerate(points):
    x = point[0]
    y = point[1]
    z = point[2]
    cur_dic = {'x': x, 'y': y, 'z': z, 'label': str(0)}
    my_list.append(cur_dic)
  cur_dic = {'x': centroid[0], 'y': centroid[1], 'z': centroid[2], 'label': str(1)}
  my_list.append(cur_dic)
  import pandas as pd
  data_frame = pd.DataFrame(my_list)  
  import plotly.express as px
  color_scheme = {"0": "yellow", "1":"red"}
  fig = px.scatter_3d(data_frame, x='z', y='x', z='y', color='label', color_discrete_map=color_scheme)
  fig.show()

# Checking if calculations work for one part instance instance

In [70]:
# all my testing chair data
chair_gt_label = chair_gt['gt_label'][2]
chair_gt_mask = chair_gt['gt_mask'][2]
chair_gt_other_mask = chair_gt['gt_other_mask'][2]
chair_gt_valid = chair_gt['gt_valid'][2]
chair_gt_points = chair_gt['pts'][2]

In [71]:
plot_cloud_instance(chair_gt_points, chair_gt_mask)

Need to gather all the points per label together

dictionary: {points: '', label: ''} -> calculate the centroid

In [72]:
my_dicts = []
for i, row in enumerate(chair_gt_mask):
  cur_inds = np.where(row)
  if len(cur_inds) > 0:
    cur_dic = {'pts': chair_gt_points[cur_inds[0]], 'label': i}
    my_dicts.append(cur_dic)

print(my_dicts)

[{'pts': array([[ 0.3060466 ,  0.78344977, -0.5374299 ],
       [ 0.10568241,  0.0135415 , -0.5233887 ],
       [-0.31424788,  0.53404814, -0.41143507],
       ...,
       [ 0.3201901 ,  0.30155712, -0.4420259 ],
       [ 0.17961667,  0.11841797, -0.42224875],
       [-0.18255904,  0.24608308, -0.41517654]], dtype=float32), 'label': 0}, {'pts': array([[ 0.56818134,  0.24550588,  0.1514212 ],
       [ 0.58026433,  0.23053606, -0.15853138],
       [ 0.54170775,  0.01001512, -0.03871135],
       ...,
       [ 0.536406  ,  0.24395926,  0.00579327],
       [ 0.5370357 ,  0.26023144, -0.01246936],
       [ 0.49432695, -0.23176825, -0.05597728]], dtype=float32), 'label': 1}, {'pts': array([[-0.5134022 ,  0.26681504,  0.00866362],
       [-0.4754287 , -0.06422284, -0.06743047],
       [-0.39670172,  0.22244571, -0.20465465],
       ...,
       [-0.4522235 ,  0.25533944, -0.14435968],
       [-0.42190713,  0.22252165, -0.02814194],
       [-0.4171923 , -0.20169815, -0.10441805]], dtype=float32)

In [73]:
centroid_1 = calculate_centroid_average(my_dicts[0]['pts'])

In [74]:
centroid_1

array([ 0.03490844,  0.31253564, -0.45173943], dtype=float32)

In [75]:
centroid_2 = calculate_centroid_middle(my_dicts[0]['pts'])

In [76]:
centroid_2

array([ 0.03313416,  0.25731921, -0.42392191])

In [77]:
plot_centroid(my_dicts[0]['pts'], centroid_1)

In [78]:
plot_centroid(my_dicts[0]['pts'], centroid_2)

# Calculating centroids of the whole object

In [93]:
# need to turn my data into a dict {'points': [], 'instance_label': ''} for easier calcuations
def to_dict(gt_mask, points):
  my_dicts = []
  for i, row in enumerate(gt_mask):
    cur_inds = np.where(row)
    if len(cur_inds[0]) > 0:
      cur_dic = {'pts': points[cur_inds[0]], 'label': i}
      my_dicts.append(cur_dic)
  return my_dicts

In [94]:
test_dict = to_dict(chair_gt_mask, chair_gt_points)
test_dict

[{'label': 0, 'pts': array([[ 0.3060466 ,  0.78344977, -0.5374299 ],
         [ 0.10568241,  0.0135415 , -0.5233887 ],
         [-0.31424788,  0.53404814, -0.41143507],
         ...,
         [ 0.3201901 ,  0.30155712, -0.4420259 ],
         [ 0.17961667,  0.11841797, -0.42224875],
         [-0.18255904,  0.24608308, -0.41517654]], dtype=float32)},
 {'label': 1, 'pts': array([[ 0.56818134,  0.24550588,  0.1514212 ],
         [ 0.58026433,  0.23053606, -0.15853138],
         [ 0.54170775,  0.01001512, -0.03871135],
         ...,
         [ 0.536406  ,  0.24395926,  0.00579327],
         [ 0.5370357 ,  0.26023144, -0.01246936],
         [ 0.49432695, -0.23176825, -0.05597728]], dtype=float32)},
 {'label': 2, 'pts': array([[-0.5134022 ,  0.26681504,  0.00866362],
         [-0.4754287 , -0.06422284, -0.06743047],
         [-0.39670172,  0.22244571, -0.20465465],
         ...,
         [-0.4522235 ,  0.25533944, -0.14435968],
         [-0.42190713,  0.22252165, -0.02814194],
         [-0.41

In [97]:
def calc_centroids_average(gt_mask, points):
  inst_lab_dict = to_dict(gt_mask, points)
  with_centroids = []
  for instance in inst_lab_dict:
    center_points = calculate_centroid_average(instance['pts'])
    cur_dict = {'label': instance['label'], 'centroid': center_points, 'pts': instance['pts']}
    with_centroids.append(cur_dict)
  return with_centroids

In [98]:
calc_centroids_average(chair_gt_mask, chair_gt_points)

[{'centroid': array([ 0.03490844,  0.31253564, -0.45173943], dtype=float32),
  'label': 0,
  'pts': array([[ 0.3060466 ,  0.78344977, -0.5374299 ],
         [ 0.10568241,  0.0135415 , -0.5233887 ],
         [-0.31424788,  0.53404814, -0.41143507],
         ...,
         [ 0.3201901 ,  0.30155712, -0.4420259 ],
         [ 0.17961667,  0.11841797, -0.42224875],
         [-0.18255904,  0.24608308, -0.41517654]], dtype=float32)},
 {'centroid': array([ 0.48121223,  0.05401397, -0.04072138], dtype=float32),
  'label': 1,
  'pts': array([[ 0.56818134,  0.24550588,  0.1514212 ],
         [ 0.58026433,  0.23053606, -0.15853138],
         [ 0.54170775,  0.01001512, -0.03871135],
         ...,
         [ 0.536406  ,  0.24395926,  0.00579327],
         [ 0.5370357 ,  0.26023144, -0.01246936],
         [ 0.49432695, -0.23176825, -0.05597728]], dtype=float32)},
 {'centroid': array([-0.41800764,  0.06017677, -0.04051073], dtype=float32),
  'label': 2,
  'pts': array([[-0.5134022 ,  0.26681504,  0.008

In [99]:
def calc_centroids_middle(gt_mask, points):
  inst_lab_dict = to_dict(gt_mask, points)
  with_centroids = []
  for instance in inst_lab_dict:
    center_points = calculate_centroid_average(instance['pts'])
    cur_dict = {'label': instance['label'], 'centroid': center_points, 'pts': instance['pts']}
    with_centroids.append(cur_dict)
  return with_centroids

In [100]:
calc_centroids_middle(chair_gt_mask, chair_gt_points)

[{'centroid': array([ 0.03490844,  0.31253564, -0.45173943], dtype=float32),
  'label': 0,
  'pts': array([[ 0.3060466 ,  0.78344977, -0.5374299 ],
         [ 0.10568241,  0.0135415 , -0.5233887 ],
         [-0.31424788,  0.53404814, -0.41143507],
         ...,
         [ 0.3201901 ,  0.30155712, -0.4420259 ],
         [ 0.17961667,  0.11841797, -0.42224875],
         [-0.18255904,  0.24608308, -0.41517654]], dtype=float32)},
 {'centroid': array([ 0.48121223,  0.05401397, -0.04072138], dtype=float32),
  'label': 1,
  'pts': array([[ 0.56818134,  0.24550588,  0.1514212 ],
         [ 0.58026433,  0.23053606, -0.15853138],
         [ 0.54170775,  0.01001512, -0.03871135],
         ...,
         [ 0.536406  ,  0.24395926,  0.00579327],
         [ 0.5370357 ,  0.26023144, -0.01246936],
         [ 0.49432695, -0.23176825, -0.05597728]], dtype=float32)},
 {'centroid': array([-0.41800764,  0.06017677, -0.04051073], dtype=float32),
  'label': 2,
  'pts': array([[-0.5134022 ,  0.26681504,  0.008

In [101]:
def plot_centroid_object(object_dict):
  my_list = []
  for instance in object_dict:
    cur_center = {'x': instance['centroid'][0], 'y': instance['centroid'][1], 'z': instance['centroid'][2], 'label': 'centroid'}
    my_list.append(cur_center)
    for point in instance['pts']:
      x = point[0]
      y = point[1]
      z = point[2]
      cur_dic = {'x': x, 'y': y, 'z': z, 'label': str(instance['label'])}
      my_list.append(cur_dic)
  import pandas as pd
  data_frame = pd.DataFrame(my_list)  
  import plotly.express as px
  fig = px.scatter_3d(data_frame, x='z', y='x', z='y', color='label')
  fig.show()

In [103]:
plot_centroid_object(calc_centroids_middle(chair_gt_mask, chair_gt_points))

In [104]:
plot_centroid_object(calc_centroids_average(chair_gt_mask, chair_gt_points))

# Calculate centroids of all the objects in chair-01 train-00

In [92]:
all_chair_gt_masks = chair_gt['gt_mask']
all_chair_points = chair_gt['pts']

(1024, 10000, 3)

In [108]:
all_chair_gt_masks.shape

(1024, 200, 10000)

In [114]:
all_chair_points.shape

(1024, 10000, 3)

In [115]:
all_objects = []

for i, obj in enumerate(all_chair_gt_masks):
  cur_dict = calc_centroids_average(obj, all_chair_points[i])
  all_objects.append(cur_dict)

In [118]:
len(all_objects)

1024

# Save centroids to h5 file

Add 'centroids' to the h5 file format in a size of 200x3 (200 instance parts, each part can have one centroid) -> if no centroid [-1, -1, -1].

In [119]:
new_file = h5py.File("test-00.h5", "w")

Turn my dicts to 1024x200x3 of centroids

In [120]:
centroids = np.array([np.array([-1., -1., -1.]) for i in range(200)])

In [121]:
all_centroids = np.array([centroids for i in range(len(all_objects))])

In [122]:
all_centroids.shape

(1024, 200, 3)

In [126]:
for i, cnt in enumerate(all_centroids):
  for instance in all_objects[i]:
    cnt[instance['label']] = instance['centroid']

In [128]:
centroids_dset = new_file.create_dataset("centroids", data=all_centroids)

In [129]:
centroids_dset = new_file.create_dataset("gt_label", data=chair_gt['gt_label'])

In [130]:
centroids_dset = new_file.create_dataset("gt_mask", data=chair_gt['gt_mask'])

In [131]:
centroids_dset = new_file.create_dataset("gt_other_mask", data=chair_gt['gt_other_mask'])

In [132]:
centroids_dset = new_file.create_dataset("gt_valid", data=chair_gt['gt_valid'])

In [133]:
centroids_dset = new_file.create_dataset("pts", data=chair_gt['pts'])

In [134]:
new_file.close()

In [136]:
trying = h5py.File("test-00.h5")

In [138]:
trying.keys()

<KeysViewHDF5 ['centroids', 'gt_label', 'gt_mask', 'gt_other_mask', 'gt_valid', 'pts']>